In [1]:
from arcgis.gis import GIS
from arcgis.geometry import Polygon
from sgc import sgccat20lib

In [4]:
PARCEL_SERVICE_NAME = "PARCEL_SERVICE_NAME"
sgccat20lib.profile_storaged = "sgc20_instalacionsgcladm"
gis = sgccat20lib.gisLogin()
print(gis.url)
dictParams = sgccat20lib.ObtenerParametros()
parcel_service = dictParams[PARCEL_SERVICE_NAME]
urlParcelService = gis.content.get(parcel_service).url
gLayers = sgccat20lib.getLayers()

https://instalacionsgcladm.esri.co/portal


### Consultar primeros 10 predios:

In [ ]:
where = "upper(estado_predio) = upper('Activo') and condicion_predio = 'PH_Unidad_Predial'"
kwargs = { "return_geometry":True, "result_record_count":10 }
predioCapa = sgccat20lib.get_features_layertable_where(
    gis,
    "{0}/{1}".format(urlParcelService, gLayers["PREDIO"]["ID"]),
    where,
    "numero_predial,globalid,objectid,condicion_predio",
    None,
    [gLayers["RELACIONES"]["predio_terreno"],gLayers["RELACIONES"]["predio_unidadconstruccion"]],
    ["*","*"],
    **kwargs)
# print(predioCapa)
if predioCapa and predioCapa.get("Relationships"):
    rel_terreno = next(iter(filter(lambda x: x["name"].upper()==gLayers["RELACIONES"]["predio_terreno"].upper(), predioCapa["Relationships"])), None)
    if len(rel_terreno["features"]) > 0:
        for row in rel_terreno["features"]:
            print(row["attributes"]["name"])
if predioCapa and predioCapa.get("Relationships"):
    rel_terreno = next(iter(filter(lambda x: x["name"].upper()==gLayers["RELACIONES"]["predio_unidadconstruccion"].upper(), predioCapa["Relationships"])), None)
    if len(rel_terreno["features"]) > 0:
        for row in rel_terreno["features"]:
            print(row["attributes"]["name"])

In [5]:
codigopredioinicial = "540010001000000020009000000000"
where = "numero_predial='{}'".format(codigopredioinicial)
kwargs = { "return_geometry": True }
predioCapa = sgccat20lib.get_features_layertable_where(
    gis,
    "{0}/{1}".format(urlParcelService, gLayers["PREDIO"]["ID"]),
    where,
    "globalid,objectid,condicion_predio",
    None,
    [gLayers["RELACIONES"]["predio_terreno"]],
    [gLayers["TERRENO"]["FIELDS"]],
    **kwargs)
if predioCapa and predioCapa.get("Relationships"):
    rel_terreno = next(iter(filter(lambda x: x["name"].upper()==gLayers["RELACIONES"]["predio_terreno"].upper(), predioCapa["Relationships"])), None)
    LOI = None
    if len(rel_terreno["features"]) > 0:
        spatialReference = rel_terreno["spatialReference"]
        for row in rel_terreno["features"]:
            print(row["attributes"]["area_terreno"])
            geometria = row["geometry"]
            geometria["spatialReference"] = spatialReference
            if LOI is None:
                LOI = Polygon(geometria)
            else:
                LOIX = Polygon(geometria)
                LOI = LOI.union(LOIX)
    print(LOI)

268750
{'rings': [[[5066016.7679, 2470462.3167000003], [5065949.0809, 2470407.1657000002], [5065783.2987, 2470303.0024], [5065745.3211, 2470256.7546], [5065679.4016, 2470180.036900001], [5065577.681000001, 2470071.6878999993], [5065566.3408, 2470049.5572], [5065454.054500001, 2470079.5197], [5065350.562000001, 2470105.9865000015], [5065276.3617, 2470115.7414], [5065248.8144000005, 2470117.444800001], [5065245.4489, 2470144.9112000004], [5065242.988500001, 2470171.712300001], [5065221.438100001, 2470289.3523999993], [5065195.387800001, 2470373.1631000005], [5065171.685500001, 2470453.4879], [5065157.3935, 2470516.3712000013], [5065154.7871, 2470573.4817999993], [5065166.192100001, 2470625.9968000003], [5065190.4429, 2470673.9104999993], [5065217.0891, 2470714.299799999], [5065342.0135, 2470714.9013000005], [5065601.4845, 2470728.509299999], [5065702.7625, 2470733.720899999], [5065818.2734, 2470739.664999999], [5066017.0865, 2470750.7339999992], [5066013.883700001, 2470715.888900001], [5

In [ ]:
loi = sgccat20lib.obtenerLOI(gLayers["PREDIO"]["LAYER"], parcel_service, gLayers["RELACIONES"]["predio_terreno"], where, None)
print(loi)

In [ ]:
import sys
import os
where = "numero_predial='{}'".format(codigopredioinicial)
ambito = "RURAL" if str(codigopredioinicial[5:7]) == "00" else "URBANO"
capa = gLayers["MANZANA"]["ID"] if ambito=="URBANO" else gLayers["VEREDA"]["ID"]
dir_files_lib = r"D:\Dev\Catastro\Repos\EditorCatastral\Geoprocesos\asignacion.GPServer"
sys.path.append(dir_files_lib)
import asignacion as asigna
condicionPropiedad = codigopredioinicial[21:22]
if condicionPropiedad == '9':
    #PH: SE UTILIZA TERRENO
    #SACAR EL PH MATRIZAL PRINCIPAL Y ASOCIADOS
    ultimos_ocho = codigopredioinicial[22:]
    if ultimos_ocho == "00000000":#ES UN PH MATRIZ
        relationship = gLayers["RELACIONES"]["predio_terreno"]
    else:#ES UNA UNIDAD PH
        relationship = gLayers["RELACIONES"]["predio_unidadconstruccion"]
elif condicionPropiedad=='5':#MEJORA NPH
    relationship = gLayers["RELACIONES"]["predio_construccion"]
else:
    relationship = gLayers["RELACIONES"]["predio_terreno"]
where_loi = "codigo='{}'".format(str(codigopredioinicial[:17]))
capa = gLayers["MANZANA"]["ID"] if ambito=="URBANO" else gLayers["VEREDA"]["ID"]
# resultLOI = asigna.obtenerLOI(parcel_service, gLayers, where, relation=relationship)
where_loi = "codigo='{}'".format(str(codigopredioinicial[0:9]))
capa = gLayers["IDS_TABLAS"]["Sector_URBANO"] if ambito=="URBANO" else gLayers["IDS_TABLAS"]["Sector_RURAL"]
resultLOI = asigna.obtenerLOI(parcel_service, gLayers, where_loi, manzanaVereda_sector=capa)
print(resultLOI)

# Predio-Relaciones:

In [ ]:
codigopredio = "080010103000001380013000000000"
numero_tramite = "08001-2021-Com63a"
versionTramite = "adminsgcv2.CON_{}".format(numero_tramite)
versionTramite = None
where_predios = "numero_predial = '{0}'".format(codigopredio)
result_predio = sgccat20lib.get_features_layertable_where(
    search_result,
    "Predio",
    where_predios,
    gLayers["PREDIO"]["FIELDS"],
    versionTramite,
    False,
    True,
    ["LC_Derecho"],
    [gLayers["tabla_derecho"]["FIELDS"]])
result_predio

# Interesados:

In [ ]:
if result_predio:
    datosPredio = result_predio["features"][0]
    result_gid = datosPredio["attributes"]["globalid"]
    rel_layer = result_predio["Relationships"][0]
    if rel_layer and len(rel_layer["objectids"]) > 0:
        OIDs_tuple = tuple(rel_layer["objectids"]) if len(rel_layer["objectids"])>1 else "('{}')".format(rel_layer["objectids"][0])
        where_derecho = "objectid in {}".format(OIDs_tuple)
        fts_derecho = sgccat20lib.get_features_layertable_where(
            search_result,
            gLayers["tabla_derecho"]["LAYER"],
            where_derecho,
            gLayers["tabla_derecho"]["FIELDS"],
            versionTramite,
            gLayers["tabla_derecho"]["IS_LAYER"])
        featuresDerecho = fts_derecho["features"]
        index = 0
        for rowDerecho in featuresDerecho:
            featuresInteresados = []
            if rowDerecho["attributes"]["lc_interesado_guid"]:
                layer = gLayers["tabla_interesados"]["LAYER"]
                out_fields2 = gLayers["tabla_interesados"]["FIELDS"]
                where = "globalid = '{}'".format(rowDerecho["attributes"]["lc_interesado_guid"])
                fts_interesado = sgccat20lib.get_features_layertable_where(search_result, layer, where, out_fields2, versionTramite, gLayers["tabla_interesados"]["IS_LAYER"])
                featuresInteresados = fts_interesado["features"] if fts_interesado else []
                del fts_interesado, layer, out_fields2, where
            elif rowDerecho["attributes"]["lc_agrupacioninteresado_guid"]:
                layer = gLayers["tabla_miembros"]["LAYER"]
                out_fields2 = gLayers["tabla_miembros"]["FIELDS"]
                out_fields_rels = [gLayers["tabla_interesados"]["FIELDS"]]
                where = "lc_agrupacioninteresados_guid = '{}'".format(rowDerecho["attributes"]["lc_agrupacioninteresado_guid"])
                fts_interesado = sgccat20lib.get_features_layertable_where(search_result, layer, where, out_fields2, versionTramite, False, True, ["LC_Interesado"], out_fields_rels)
                if fts_interesado and len(fts_interesado["Relationships"]) > 0:
                    featuresInteresados = fts_interesado["Relationships"][0]["features"]
                del layer, out_fields2, where, out_fields_rels
                print("Miembros:")
                print(fts_interesado['features'])

            for interesado in featuresInteresados:
                index += 1
                print("INTERESADO {}: {}".format(index, str(interesado['attributes'])))
        print("Total Interesados: {}".format(len(featuresInteresados)))
       # print(fts_interesado['features'])

# Fuente Administrativa:

In [ ]:
if result_predio:
    datosPredio = result_predio["features"][0]
    rel_layer = next(iter(filter(lambda x: x["name"].upper() == gLayers["RELACIONES"]["predio_derecho"].upper(), result_predio["Relationships"])), None)
    if rel_layer and len(rel_layer["objectids"]) > 0:
        OIDs_tuple = tuple(rel_layer["objectids"]) if len(rel_layer["objectids"])>1 else "('{}')".format(rel_layer["objectids"][0])
        where_derecho = "objectid in {}".format(OIDs_tuple)
        res_FAdmin = sgccat20lib.get_features_layertable_where(
            search_result,
            gLayers["tabla_derecho"]["LAYER"],
            where_derecho,
            gLayers["tabla_derecho"]["FIELDS"],
            versionTramite,False,True,
            [gLayers["RELACIONES"]["derecho_fuenteadmin"]],
            [gLayers["tabla_fuenteadministrativa"]["FIELDS"]])
        print("Total Fuente Admin: {}".format(len(res_FAdmin['Relationships'][0]['features'])))
        index = 0
        for row in res_FAdmin['Relationships'][0]['features']:
            index += 1
            print("FAdmin {}: {}".format(index, str(row['attributes'])))

# Col Miembros:

In [ ]:
if len(res_Derecho['Relationships'][0]['features']) > 0:
    objectids_interesado = []
    for interesado in res_Derecho['Relationships'][0]['features']:
        objectids_interesado.append(interesado["attributes"]["objectid"])
    if len(objectids_interesado) > 0:
        res_miembros = sgccat20lib.get_features_layertable(search_result,gLayers["tabla_interesados"]["LAYER"],objectids_interesado,gLayers["tabla_miembros"]["FIELDS"],versionTramite,False,True,[gLayers["RELACIONES"]["interesado_miembros"]])
        print("Total res_miembros: {}".format(len(res_miembros['Relationships'][0]['features'])))
        index = 0
        for miembro in res_miembros['Relationships'][0]['features']:
            index += 1
            print("MIEMBRO {}: {}".format(index, str(miembro)))

In [ ]:
datos_derecho = {
    "tipodocumento": "Cedula_Ciudadania",
    "nrodocumento": "string",
    "primernombre": "string",
    "segundonombre": "string",
    "primerapellido": "string",
    "segundoapellido": "string",
    "razonsocial": "string",
    # "tipoderecho": "Dominio",
    # "fraccionderecho": 0,
    "numeradorparticipacion": 0,
    "denominadorparticipacion": 0,
    "participacion": 0
}
if datos_derecho:
    #Validacion fraccion_derecho * tipo:
    if "tipoderecho" in datos_derecho and "fraccionderecho" in datos_derecho:
        print("Existe TD y FD")
    elif "tipoderecho" in datos_derecho and "fraccionderecho" not in datos_derecho:
        print("Existe TD y no FD")
    elif "tipoderecho" not in datos_derecho and "fraccionderecho" in datos_derecho:
        print("No Existe TD y FD si")
    print("NO pasa nada")

In [ ]:
atributos_lista = [{'participacion': 1, "num": 2, 'lc_interesado_guid': '{BC457569-9691-4D1F-AB91-88748635BE92}'}]
# fts_colmiembros = [{'attributes': {'participacion': 1, 'lc_interesado_guid': '{BC457569-9691-4D1F-AB91-88748635BE92}', 'objectid': 2811}}, {'attributes': {'participacion': 1, 'lc_interesado_guid': '{BC457569-9691-4D1F-AB91-88748635BE92}', 'objectid': 2811}}, {'attributes': {'participacion': 1, 'lc_interesado_guid': '{BC457569-9691-4D1F-AB91-88748635BE92}', 'objectid': 2811}}, {'attributes': {'participacion': 1, 'lc_interesado_guid': '{BC457569-9691-4D1F-AB91-88748635BE92}', 'objectid': 2811}}]
fts_colmiembros = [{'attributes': {'lc_interesado_guid': '{BC457569-9691-4D1F-AB91-88748635BE92}', 'objectid': 2811}},
{'attributes': {'lc_interesado_guid': '{BC457569-9691-4D1F-AB91-88748635BE92}', 'objectid': 2812}},
{'attributes': {'lc_interesado_guid': '{BC457569-9691-4D1F-AB91-88748635BE92}', 'objectid': 2813}},
{'attributes': {'lc_interesado_guid': '{BC457569-9691-4D1F-AB91-88748635BE92}', 'objectid': 2814}}]
for row in atributos_lista:
    print(str(row))
    updates = list(filter(lambda x: x["attributes"]["lc_interesado_guid"]==row["lc_interesado_guid"], fts_colmiembros))
    # print(str(colmiembros_interesado[0]))
    for item in updates:
        for attribute in row:
            item["attributes"][attribute] = row[attribute]
    # print(str(updates))
    if len(updates) > 0:
        for row in updates:
            del row["attributes"]["lc_interesado_guid"]
            # break
    print(str(updates))
    participacion = 0
    for row in updates:
        participacion += row["attributes"]["participacion"]
    if participacion > 1:
        print("La suma total de participacion de interesados no puede ser superior a 1. Suma total participacion(es): {}".format(participacion))

In [ ]:
where_predios = "numero_predial = '{0}'".format(codigopredio)
result_predio = sgccat20lib.get_predios_where(search_result, where_predios, "globalid,numero_predial", versionTramite)
if len(result_predio) > 0:
    datosPredio = result_predio[0]
    print(datosPredio)
    if len(datosPredio["features"]) > 0:
        result_gid = datosPredio["features"][0].attributes["globalid"]
        print(result_gid)

In [ ]:
tablaConsulta = {"derecho":{"objectids":['1']}}
OIDs_derecho = [f for f in tablaConsulta["derecho"]["objectids"]]
OIDs_tuple = tuple(OIDs_derecho) if len(OIDs_derecho)>1 else "({})".format(OIDs_derecho[0])
where_derecho = "objectid in {}".format(OIDs_tuple)
print(where_derecho)

In [ ]:
search_result.url